<h2> Part 1: Exploring and  preparing the data for analysis </h2> 
 We start with the import of packages we will eventually need. Furthermore, we import the datasets and start with exploring and preparing the data for further analysis.
 
 Firstly, we import the packages we need in the notebook for the analysis of the two uploaded datasets. Afterthat,  we start exploring the datasets to get a brief overview of the available data. In order to make a proper analysis we preparing the datasets by cleaning and reordering the different collums and rows.

In [ ]:
movies = pd.read_csv('../input/tmdb_5000_movies.csv')
credits = pd.read_csv('../input/tmdb_5000_credits.csv')



<h4> What does the data look like?</h4>
We'll start with taking a look at the movies data frame to see what we've got the work with.


In [ ]:
movies.head()

<h4> What are remarkable observations/findings if we take a quick look at the dataframe?</h4>
The first thing we notice is that the columns are a strange order to take a fine look at the data. A preferable first column of this data frame, would, for example, be the title of the movie and not the movie's budget. 

Furthermore,  we notice that the columns 'genres', 'keywords', 'production_companies', 'production_countries' and 'spoken_languages' are of the dictionary type, so right now they are quite hard to read, but later on we will find a way to work with them.

Amongst the numerical columns, there's a movie budget, a movie ID, popularity, revenue, runtime, a vote average and the amount of votes a movie has received.  Besides the fact that the ID column is numerical, it is also not of interest for making predictions about. 

For now, we leave this data frame as it is and we'll take a quick look at the other one.

In [ ]:
credits.head()

<h4> What are remarkable observations/findings if we take a quick look at the dataframe?</h4>
The data frame credits has way fewer columns. The cast and crew might be interesting later on. Since this data frame contains only two extra columns, we'll try to merge it with the  movies data frame. If tthe collumns have the same value, we can just concatenate the data frames,. We use the following function to see if in both data frames every row is about the same movie.

In [ ]:
(credits['title']==movies['title']).describe()

We can conclude that every row in the collumn title of the credits data base has the same value  as the same row in the collumn title of the  movies data base. To prevent getting duplicate columns, we'll remove the movie_id and title column from the credits data frame and concatenate them. This results in the following combined dataframe:

In [ ]:
del credits['title']
del credits['movie_id']
movie_df = pd.concat([movies, credits], axis=1)
movie_df.head()

The collumns are still in a stange order if we take a quick look at the dataframe. Moreover, columns like homepage, id and status aren't really interesting for us. Therefore, we choose the interesting columns (in our case), put them in a nice order and create a new data frame.


In [ ]:
newCols = ['title','release_date','popularity','vote_average','vote_count',
           'budget','revenue','genres','keywords','cast','crew','tagline', 'runtime', 'production_companies', 
           'production_countries', 'status']

df2 = movie_df[newCols]
df2.head()

<h4>What can the numerical collumns tell us if we make a quick analysis?</h4>

In [ ]:
df2.describe().round()

Note that runtime consists of two empty values, before we can really work with our data frame, we need to solve this. We use an imputer for this, which replaces the missing value by a computed average.

In [ ]:
my_imputer = Imputer()

temp=df2
X2 = my_imputer.fit_transform(df2[['runtime']])
df2['runtime'] = X2
df2.describe().round()

<h4>How are all numerical variables distributed?</h4>
<span style="color:red">We firstly devide the vote average into four classes (low, medium-low, medium-high, high) .....


In [ ]:
#df2['vote_classes'] = pd.cut(df2['vote_average'],10, labels=["1", "2","3","4","5","6","7","8","9","10"])
df2['vote_classes'] = pd.cut(df2['vote_average'],4, labels=["low", "medium-low","medium-high","high"])

In [ ]:
df2['log_budget'] = np.log(df2['budget'])
df2['log_popularity'] = np.log(df2['popularity'])
df2['log_vote_average'] = np.log(df2['vote_average'])
df2['log_vote_count'] = np.log(df2['vote_count'])
df2['log_revenue']= np.log(df2['revenue'])
df2['log_runtime']= np.log(df2['runtime'])
df3=df2[df2.columns[-5:]]

#df3.replace([np.inf, -np.inf], np.nan).dropna(axis=1)
df3=df3[df3.replace([np.inf, -np.inf], np.nan).notnull().all(axis=1)]
df3=df3.dropna(axis=1)
#df3[~df3.isin([np.nan, np.inf, -np.inf]).any(1)]


In [ ]:
from pandas.plotting import scatter_matrix
scatter_matrix(df3,alpha=0.2, figsize=(20, 20), diagonal='kde')

<span style="color:red">If we take a look at the distrbutions we can conclude that ....

In [ ]:
Early_df = df2[df2.columns[0:16]]
Early_df.head()

Note that everything is quite skewed. We'll try getting more in depth into this later.

<span style="color:red">Snap het doel van de  laatste tabel niet helemaal en wat je met deze opmerking bedoelt.

<h4>How are the genres and revenue related over the years if we remove the outliers from revenue and budget?</h4>
<span style="color:red">Nog een heatmap toevoegen voor genres en profit waarbij de outliners van budget en revenue zijn verwijderd.

<h4>Which genre is best scoring in the catagories mean votes average, mean budget and mean revenue?

In [ ]:
mean_per_keyword.sort_values('mean_vote_average', ascending=False).head()

In [ ]:
mean_per_keyword.sort_values('mean_budget', ascending=False).head()

In [ ]:
mean_per_keyword.sort_values('mean_revenue', ascending=False).head()

We want to create the tables above again, but then for only the 50 most occuring keywords. Of course it's cool to see the keywords 'hobbit' and 'school of witchcraft' as high-revenue-keywords, but they probably don't occur outside of he Lord of the Rings and Harry Potter movies, so they're actually not that interesting. We start by showing the 50 most occuring keywords in a bar plot.

In [ ]:
fig = plt.figure(1, figsize=(18,13))
trunc_occurences = new_keyword_occurences[0:50]
# LOWER PANEL: HISTOGRAMS
ax2 = fig.add_subplot(2,1,2)
y_axis = [i[1] for i in trunc_occurences]
x_axis = [k for k,i in enumerate(trunc_occurences)]
x_label = [i[0] for i in trunc_occurences]
plt.xticks(rotation=85, fontsize = 15)
plt.yticks(fontsize = 15)
plt.xticks(x_axis, x_label)
plt.ylabel("Nb. of occurences", fontsize = 18, labelpad = 10)
ax2.bar(x_axis, y_axis, align = 'center', color='g')
#_______________________
plt.title("Keywords popularity",bbox={'facecolor':'k', 'pad':5},color='w',fontsize = 25)
plt.show()

We now create a data frame with average movie scores, budget and revenue for the most occuring keywords.

In [ ]:
Df1 = pd.DataFrame(trunc_occurences)
Df2 = mean_per_keyword
result = Df1.merge(Df2, left_on=0, right_on=0, how='inner')

In [ ]:
result = result.rename(columns ={0:'keyword', 1:'occurences'})

In [ ]:
result.sort_values('mean_vote_average', ascending= False)

Let's try to visualize this table a little bit clearer. We want to see which keywords pop out, so let's make a few plots.

In [ ]:
result['mean_vote_average'].mean()

In [ ]:
import matplotlib.pyplot as plt

ax = result.plot.bar(x = 'keyword', y='mean_vote_average', title="mean vote average",
                     figsize=(15,4), legend=True, fontsize=12, color='green', label = "mean vote average")
ax.set_ylim(5, 8)
ax.axhline(y=result['mean_vote_average'].mean(),c="blue",linewidth=0.5, label='mean')
ax.legend()
plt.show()


In [ ]:
import matplotlib.pyplot as plt

ax = result.plot.bar(x = 'keyword', y='mean_budget', title="mean budget",
                     figsize=(15,4), legend=True, fontsize=12, color='green', label="mean budget")
ax.axhline(y=result['mean_budget'].mean(),c="blue",linewidth=0.5, label='mean')
ax.legend()
plt.show()


In [ ]:
result.sort_values('mean_budget').head()

Now this is interesting. The keyword with by far the highest average rating - Serial Killer - also has by far the lowest average budget. Also note that superhero movies score below average, but have a highly above average budget. Let's take a look at the revenue:

In [ ]:
ax = result.plot.bar(x = 'keyword', y='mean_revenue', title="mean revenue",
                     figsize=(15,4), legend=True, fontsize=12, color='green', label="mean revenue")
ax.axhline(y=result['mean_revenue'].mean(),c="blue",linewidth=0.5, label='mean')
ax.legend()
plt.show()

So superhero movies do have a high revenue and serial killer movies do not. Let's take a look at the differences:

In [ ]:
result['profit'] = result['mean_revenue'] - result['mean_budget']
result.head()

In [ ]:
ax = result.plot.bar(x = 'keyword', y='profit', title="profit",
                     figsize=(15,4), legend=True, fontsize=12, color='green', label="profit")
ax.axhline(y=result['profit'].mean(),c="blue",linewidth=0.5, label='mean')
ax.legend()
plt.show()

What's also notable is that despite the popularity of female directors and independent movies, they do not have high scores on either revenue or vote_average.

<h2> Part 4: Analyse Actors

Part 4 contains is about the actors of the movies. We analyse which actors made the most revenue, resulted in a high IMDB score, and played in movies with the highest budgets. We also made a graph that contains information about the average budget and every revenue per actor. Lastly, we made a clear overview for all the movies Morgan Freeman played in and what the IMDB scores where. The main purpuse of this part is getting more insight in the data and answering the following questions:  <br> 
    - Which actors played in movies with the highest revenue/highest budget/ highest IMDB score?
    


A previous version of this dataset only contained the top three actors per movie. Since we only want to analyze the most important actors of a movie and since the old dataset was a bit more suited to do that, we convert the dataset back to its previous state using Sohier Dane's method.

We can now see that the dataframe is simplified.

In [ ]:
df.head()

In [ ]:
df3 = df # We store a copy of the dataframe for later use

Next, we delete all the columns we won't be needing for this analysis.

In [ ]:
columns = ['homepage', 'plot_keywords', 'language', 'overview', 'popularity', 'tagline',
           'original_title', 'num_voted_users', 'country', 'spoken_languages', 'duration',
          'production_companies', 'production_countries', 'status']

df = df.drop(columns, axis=1)

We are interested in the same descriptives for the actors, as we were for keywords and the genres. To do that, we first have to, once again, restructure the dataframe.

We first create a seperate dataframe for each of the three actors, after which we can combine them to get one dataframe with all three types of actor.

In [ ]:
liste_genres = set()
for s in df['genres'].str.split('|'):
    liste_genres = set().union(s, liste_genres)
liste_genres = list(liste_genres)
liste_genres.remove('')

In [ ]:
df_reduced = df[['actor_1_name', 'vote_average',
                 'title_year', 'movie_title', 'gross', 'budget']].reset_index(drop = True)
for genre in liste_genres:
    df_reduced[genre] = df['genres'].str.contains(genre).apply(lambda x:1 if x else 0)

df_reduced2 = df[['actor_2_name', 'vote_average',
                 'title_year', 'movie_title', 'gross', 'budget']].reset_index(drop = True)
for genre in liste_genres:
    df_reduced2[genre] = df['genres'].str.contains(genre).apply(lambda x:1 if x else 0)

df_reduced3 = df[['actor_3_name', 'vote_average',
                 'title_year', 'movie_title', 'gross', 'budget']].reset_index(drop = True)
for genre in liste_genres:
    df_reduced3[genre] = df['genres'].str.contains(genre).apply(lambda x:1 if x else 0)


Next, we combine the three dataframes.

In [ ]:
df_reduced = df_reduced.rename(columns={'actor_1_name': 'actor'})
df_reduced2 = df_reduced2.rename(columns={'actor_2_name': 'actor'})
df_reduced3 = df_reduced3.rename(columns={'actor_3_name': 'actor'})

total = [df_reduced, df_reduced2, df_reduced3]
df_total = pd.concat(total)
df_total.head()

We compute averages for all actors in two categories: vote_average and title_year. We also compute an actors favorite genre.

In [ ]:
df_actors = df_total.groupby('actor').mean()
df_actors.loc[:, 'favored_genre'] = df_actors[liste_genres].idxmax(axis = 1)
df_actors.drop(liste_genres, axis = 1, inplace = True)
df_actors = df_actors.reset_index()

We expect the dataframe to contain a lot of actors that have only a single observation. These observation are likely to cause outliers if these observations are very extreme. We delete all actors that are linked to less than 10 movies in our dataframe.

In [ ]:
df_appearance = df_total[['actor', 'title_year']].groupby('actor').count()
df_appearance = df_appearance.reset_index(drop = True)
selection = df_appearance['title_year'] > 9
selection = selection.reset_index(drop = True)
most_prolific = df_actors[selection]

Now that we have a clear dataframe, let us show some descriptive statistics. We first sort the dataframe on all the different attributes from highest to lowest.

In [ ]:
most_prolific.sort_values('vote_average', ascending=False).head()

In [ ]:
most_prolific.sort_values('gross', ascending=False).head()

In [ ]:
most_prolific.sort_values('budget', ascending=False).head()

Looks like Sir Ian McKellen has had quite a career. He came out on top on all three of our attributes. He plays in the movies with the highsest budget, but returns this with the highest average revenues. It makes sense that these enormous budgets lead to good movies. This is reflected by him having the highest average score on IMDB.

We can now develop several plots to analyze our actors. Let us start by plotting the average budget per actor and the average revenue per actor.

In [ ]:
genre_count = []
for genre in liste_genres:
    genre_count.append([genre, df_reduced[genre].values.sum()])
genre_count.sort(key = lambda x:x[1], reverse = True)
labels, sizes = zip(*genre_count)
labels_selected = [n if v > sum(sizes) * 0.01 else '' for n, v in genre_count]
reduced_genre_list = labels[:19]
trace=[]
for genre in reduced_genre_list:
    trace.append({'type':'scatter',
                  'mode':'markers',
                  'y':most_prolific.loc[most_prolific['favored_genre']==genre,'gross'],
                  'x':most_prolific.loc[most_prolific['favored_genre']==genre,'budget'],
                  'name':genre,
                  'text': most_prolific.loc[most_prolific['favored_genre']==genre,'actor'],
                  'marker':{'size':10,'opacity':0.7,
                            'line':{'width':1.25,'color':'black'}}})
layout={'title':'Actors favored genres',
       'xaxis':{'title':'mean year of activity'},
       'yaxis':{'title':'mean score'}}
fig=Figure(data=trace,layout=layout)
pyo.iplot(fig)

We can also use this data to highlight single actors. Let us take a look at actors for who we have data of more than 20 movies.

In [ ]:
selection = df_appearance['title_year'] > 20
most_prolific = df_actors[selection]
most_prolific

So let's have a look at Morgan Freeman. We would like to have a clear overview of all the movies he played in and what his movies scored on IMDB. We can do this using a polar chart.

In [ ]:
class Trace():
    #____________________
    def __init__(self, color):
        self.mode = 'markers'
        self.name = 'default'
        self.title = 'default title'
        self.marker = dict(color=color, size=110,
                           line=dict(color='white'), opacity=0.7)
        self.r = []
        self.t = []
    #______________________________
    def set_color(self, color):
        self.marker = dict(color = color, size=110,
                           line=dict(color='white'), opacity=0.7)
    #____________________________
    def set_name(self, name):
        self.name = name
    #____________________________
    def set_title(self, title):
        self.na = title
    #___________________________
    def set_actor(self, actor):
        self.actor = actor
    
    #__________________________
    def set_values(self, r, t):
        self.r = np.array(r)
        self.t = np.array(t)

In [ ]:
names =['Morgan Freeman']
df2 = df_reduced[df_reduced['actor'] == 'Morgan Freeman']
total_count  = 0
years = []
imdb_score = []
genre = []
titles = []
actor = []
for s in liste_genres:
    icount = df2[s].sum()
    #__________________________________________________________________
    # Here, we set the limit to 3 because of a bug in plotly's package
    if icount > 3: 
        total_count += 1
        genre.append(s)
        actor.append(list(df2[df2[s] ==1 ]['actor']))
        years.append(list(df2[df2[s] == 1]['title_year']))
        imdb_score.append(list(df2[df2[s] == 1]['vote_average'])) 
        titles.append(list(df2[df2[s] == 1]['movie_title']))
max_y = max([max(s) for s in years])
min_y = min([min(s) for s in years])
year_range = max_y - min_y

years_normed = []
for i in range(total_count):
    years_normed.append( [360/total_count*((an-min_y)/year_range+i) for an in years[i]])

In [ ]:
color = ('royalblue', 'grey', 'wheat', 'c', 'firebrick', 'seagreen', 'lightskyblue',
          'lightcoral', 'yellowgreen', 'gold', 'tomato', 'violet', 'aquamarine', 'chartreuse', 'red')

In [ ]:
trace = [Trace(color[i]) for i in range(total_count)]
tr    = []
for i in range(total_count):
    trace[i].set_name(genre[i])
    trace[i].set_title(titles[i])
    trace[i].set_values(np.array(imdb_score[i]),
                        np.array(years_normed[i]))
    tr.append(go.Scatter(r      = trace[i].r,
                         t      = trace[i].t,
                         mode   = trace[i].mode,
                         name   = trace[i].name,
                         marker = trace[i].marker,
#                         text   = ['default title' for j in range(len(trace[i].r))], 
                         hoverinfo = 'all'
                        ))        
layout = go.Layout(
    title='Morgan Freeman movies',
    font=dict(
        size=15
    ),
    plot_bgcolor='rgb(223, 223, 223)',
    angularaxis=dict(        
        tickcolor='rgb(253,253,253)'
    ),
    hovermode='Closest',
)
fig = go.Figure(data = tr, layout=layout)
pyo.iplot(fig)

Unfortunately, plotly doesn't allow us to put hover text on the different notes. This means we can't add the movie name and year of release to all the different nodes.

### 3.5 Director    

Directors of movies are known as an important influence for the quality of a movie. This part will analyse the IMDB score per director. In this analyse we try to discover if the IMDB score of movies from high scoring directors are influenced by the budget. This brings us to the following questions: <br> 
<br> 
For directors with more than 4/ and directors with more than 15 movies: 
    - Which directors have the highest revenue per movie?
    - Which directors have the highest budget per movie?
    - Do directors with high budgets also have high revenues? 
    

We start by retrieving the copy of the database we created earlier. After that, we  analyse by computing the average per movie and total revenue of the directors. We only took into account the directors for which we have at least 4 movies as observations, to exclude extreme outliers. Not surprisingly, the top rated directors are probably directors you have heard about.

In [ ]:
#Directors are counted and compared
df = df3

def create_comparison_database(name, value, x, no_films):
    
    comparison_df = df3.groupby(name, as_index=False)
    
    if x == 'mean':
        comparison_df = comparison_df.mean()
    elif x == 'median':
        comparison_df = comparison_df.median()
    elif x == 'sum':
        comparison_df = comparison_df.sum() 
    
    # Create database with either name of directors or actors, the value being compared i.e. 'revenue',
    # and number of films they're listed with. Then sort by value being compared.
    name_count_key = df[name].value_counts().to_dict()
    comparison_df['films'] = comparison_df[name].map(name_count_key)
    comparison_df.sort_values(value, ascending=False, inplace=True)
    comparison_df[name] = comparison_df[name].map(str) + " (" + comparison_df['films'].astype(str) + ")"
   # create a Series with the name as the index so it can be plotted to a subgrid
    comp_series = comparison_df[comparison_df['films'] >= no_films][[name, value]][10::-1].set_index(name).ix[:,0]
    
    return comp_series

Two questions we want to answer are:<br>
**Which directors created the most total revenue if produced atleast 4 movies?**<br>
**Which directors created the most  average revenue  if produced at least 4 movies?**

By creating a bar chart we can get a good overview/visualition of directors with 4+ movies and the revenue. 

In [ ]:
fig = plt.figure(figsize=(18,6))

# Director_name
plt.subplot2grid((2,3),(0,0), rowspan = 2)
create_comparison_database('director_name','gross','sum', 4).plot(kind='barh', color='#006600')
plt.legend().set_visible(False)
plt.title("Total revenue for Directors with 4+ Films")
plt.ylabel("Director (no. films)")
plt.xlabel("Revenue")

plt.subplot2grid((2,3),(0,1), rowspan = 2)
create_comparison_database('director_name','gross','mean', 4).plot(kind='barh', color='#ffff00')
plt.legend().set_visible(False)
plt.title('Average revenue for Directors with 4+ Films')
plt.ylabel("Director (no. films)")
plt.xlabel("Revenue")

plt.tight_layout()

It's not very remarkable that Steven Spielberg has gained the most revenue out of the movies he has dericted due to the fact that he produced the most movies. Moreover, his name is well known as a good director. However, on average he is not in the top 10, probably because he has produced 27 movies. If we compare the two bar charts we can conclude that Peter Jackson and James Cameron score well in both charts. 

We used a bar chart to  get an better insight of the directors and the following questions: <br>
**Which What directors with on average more than 4 movies directed got access to the highest budget**<br>
**Which directors have on average the highest IMDB score if if produced at least 4 movies?**



In [ ]:
fig = plt.figure(figsize=(18,6))

# Director_name
plt.subplot2grid((2,3),(0,0), rowspan = 2)
create_comparison_database('director_name','budget','mean', 4).plot(kind='barh', color='#006600')
plt.legend().set_visible(False)
plt.title("Average budget for Directors with 4+ Filmss")
plt.ylabel("Director (no. films)")
plt.xlabel("Budget (in billons)")

plt.subplot2grid((2,3),(0,1), rowspan = 2)
create_comparison_database('director_name','vote_average','mean', 4).plot(kind='barh', color='#ffff00')
plt.legend().set_visible(False)
plt.title('Mean IMDB Score for Directors with 4+ Films')
plt.ylabel("Director (no. films)")
plt.xlabel("IMDB Score")
plt.xlim(0,10)

plt.tight_layout()

Notice how many of the directors that have a very high average budget per movie were nowhere to be seen in the revenue plot. Implying that, although they make expensive movies, they don't make the most grossing movies. Also note that a lot of high scoring directors are not found in the top ten highest budgeted directors. This implies that a big budget doesn't necessarily lead to a good, or well-received, movie. On the other hand, it shows that some directors, for instance Hayao Miyazaki, is capable of creating excellent movies without needing a very high budget.

However, it is always possible that directors with few movies were lucky. Because we took into acount evry director with 4+ movies. Therefore, for the next part we only took the directors with 15+ movies and like to know: <br>

**Are there any directors that are capable of consistently creating well-received movies, without the need for big budgets? **

To answer this question we plot the average budget next to the average score per director, for directors with at least 15 movies.

In [ ]:
fig = plt.figure(figsize=(18,6))

# Director_name
plt.subplot2grid((2,3),(0,0), rowspan = 2)
create_comparison_database('director_name','budget','mean', 10).plot(kind='barh', color='#006600')
plt.legend().set_visible(False)
plt.title("Average budget for Directors with 15+ Filmss")
plt.ylabel("Director (no. films)")
plt.xlabel("Budget")

plt.subplot2grid((2,3),(0,1), rowspan = 2)
create_comparison_database('director_name','vote_average','mean', 10).plot(kind='barh', color='#ffff00')
plt.legend().set_visible(False)
plt.title('Mean IMDB Score for Directors with 15+ Films')
plt.ylabel("Director (no. films)")
plt.xlabel("IMDB Score")
plt.xlim(0,10)

plt.tight_layout()

Now, we easily see that the two bar plots have more directors in common. Still, there are some directors who manage to create excellent movies without the need for a big budget. A funny observation is Michael Bay. While he is easily the king of budget, he is nowhere to be found in the top ten highest scoring directors.

In [ ]:
df2.head()

# Numerical Analysis

In this part we take a closer look at the numerical columns in the data frame. Which enables us to gather more information about how the data is correlated and which variables might be interesting for the prediction in later parts. The main question for this part is: 
    - How is the numerical data in our dataframe correlated? 
    
The first step is to create a dataframe which only contains numerical data. Since we want to analyse numerical data for this part only

In [ ]:
df2 = Early_df

In [ ]:
df2['log_budget'] = np.log(df2['budget'])
df2['log_popularity'] = np.log(df2['popularity'])
df2['log_vote_average'] = np.log(df2['vote_average'])
df2['log_vote_count'] = np.log(df2['vote_count'])
df2['log_revenue']= np.log(df2['revenue'])
df2['log_runtime']= np.log(df2['runtime'])
df3=df2[df2.columns[-6:]]

df3=df3[df3.replace([np.inf, -np.inf], np.nan).notnull().all(axis=1)]
df3=df3.dropna(axis=1)
df3.head()

In [ ]:
num_list = ['budget','popularity','revenue','runtime','vote_average','vote_count']
movie_num = df2[num_list]
movie_num.head()

In [ ]:
df3.head()

In [ ]:
movie_num['revenue'] .plot(kind='hist')

In [ ]:
df3['log_revenue'].plot(kind='hist')

Let's take a look at how everything is correlated:

In [ ]:
f, ax = plt.subplots(figsize=(12,10))
plt.title('Pearson Correlation of Movie Features')
sns.heatmap(movie_num.astype(float).corr(), linewidths=0.25, vmax=1.0, square=True,
           cmap="YlGnBu", linecolor='black', annot=True)

# Numerical Analysis
In this part the purpose was to find out which variables are correlated. As the graph shows there are quite some dark/blue squares which means high correlation. Therefore, we expect that we can make a decent model in the next part where we try to predict the vote count. The variables that have a higher correlation than 0.70 are: <br> 
    - Revenue and Budget, correlation of 0.73.
    - Vote count and popularity, correlation of 0.78.
    - Vote count and revenue, correaltion of 0.78. 


# Comparing different regression techniques


In this part we want to compare a few regression techniques to help us in making predictions. We'll use linear regression and random forest.<br> 
This section gives answer on the following questions: <br> 
    - Which regression technique provides the best predicitions?
    - How well are we able to predict the average votes given the budget, popularity, revenue, runtime, and vote_count? 
We start by recreating our numerical data frame.

In [ ]:
num_list = ['budget','popularity','revenue','runtime','vote_average','vote_count']
movie_num = df2[num_list]
movie_num.head()

We want the vote_average to be our target values, budget, popularity, revenue, runtime and vote_count are trainng values.

In [ ]:
training_list = ['budget','popularity','revenue','runtime','vote_count']
training = movie_num[training_list]
target = movie_num['vote_average']

In [ ]:
X = training.values
y = target.values

We split our data in a train and a test frame.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)

Now let's train a linear regression model and plot the results: \***

In [ ]:
from sklearn import linear_model
# Create linear regression object
regr = linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(X_train, y_train)

# Make predictions using the testing set
y_pred_lr = regr.predict(X_test)

In [ ]:
f = plt.figure(figsize=(10,5))
plt.scatter(X_test[:,1], y_test, s=50,label="Real vote_average");
plt.scatter(X_test[:,1], y_pred_lr,s=100, c='r',label="Predicted vote_average");
plt.ylabel("vote_average");
plt.legend(loc=2);

Now let's see what happens if we use a random forest regression model:

In [ ]:
from sklearn.ensemble import RandomForestRegressor
# Create linear regression object
rf = RandomForestRegressor(1)

# Train the model using the training sets
rf.fit(X_train, y_train)

# Make predictions using the testing set
y_pred_rf = rf.predict(X_test)

In [ ]:
f = plt.figure(figsize=(10,5))
plt.scatter(X_test[:,1], y_test, s=50,label="Real vote_average");
plt.scatter(X_test[:,1], y_pred_rf,s=100, c='r',label="Predited vote_average");
plt.ylabel("vote_average");
plt.legend(loc=2);

And let's compare them:

In [ ]:
from sklearn.metrics import mean_squared_error

error_lr = mean_squared_error(y_test,y_pred_lr)
error_rf = mean_squared_error(y_test,y_pred_rf)

print(error_lr)
print(error_rf)

In [ ]:
f = plt.figure(figsize=(10,5))
plt.bar(range(2),[error_lr,error_rf])
plt.xlabel("Classifiers");
plt.ylabel("Mean Squared Error of the vote_average");
plt.xticks(range(2),['Linear Regression','Random Forest'])
plt.legend(loc=2);

<h3> Conclusion comparing different regression techniques </h3>
This parts purpose was to found out how well we can predict the vote average and test which regression method is better. Firstly, both models estimators seem to be very decent. Which shows that we are able to predict the average votes for a movie, with the given variables, without much error. The mean squared error for random forrest is slightly higher than the mean squared error for the linear regression. So this suggest that the model of linear regression is the best model to use for our data. However, both models are able to predict with precision. 

\* https://www.kaggle.com/fabiendaniel/categorizing-actors-hands-on-plotly <br>
\** https://www.kaggle.com/diegoinacio/imdb-genre-based-analysis <br>
\*** introduction to data science, week 4, Comparison of Regression Techniques on House prediction prices.ipynb

Resources:

https://www.kaggle.com/fabiendaniel/film-recommendation-engine

https://www.kaggle.com/fabiendaniel/categorizing-actors-hands-on-plotly

https://www.kaggle.com/willacy/director-and-actor-s-total-gross-and-imdb-score